# The IPython widgets, now in IHaskell !!

It is highly recommended that users new to jupyter/ipython take the *User Interface Tour* from the toolbar above (Help -> User Interface Tour).

## The Widget Hierarchy

### Implemented so far

```
.
├── Bool
│   ├── CheckBox
│   └── ToggleButton
├── Button
├── Float
│   ├── BoundedFloat
│   │   ├── BoundedFloatText
│   │   ├── FloatProgress
│   │   └── FloatSlider
│   ├── BoundedFloatRange
│   │   └── FloatRangeSlider
│   └── FloatText
├── Image
├── Int
│   ├── BoundedInt
│   │   ├── BoundedIntText
│   │   ├── IntProgress
│   │   └── IntSlider
│   ├── BoundedIntRange
│   │   └── IntRangeSlider
│   └── IntText
├── Output
├── Selection
│   ├── Dropdown
│   ├── RadioButtons
│   ├── Select
│   ├── SelectMultiple
│   └── ToggleButtons
└── String
    ├── HTML
    ├── Latex
    ├── TextArea
    └── Text
```

### Not implemented yet

```
.
└── Box
    ├── FlexBox
    └── SelectionContainer
        ├── Accordion
        └── Tab
```

## The Widgets

### Necessary Extensions and Imports

In [1]:
{-# LANGUAGE OverloadedStrings #-}
import IHaskell.Display.Widgets

### Common widget interface

Each widget has different properties, but the surface level API is the same.

Every widget has:

1. A constructor:
    An `IO <widget>` value/function of the form `mk<widget_name>`.
2. A set of properties, which can be manipulated using `setField` and `getField`.

The `setField` and `getField` functions have nasty type signatures, but they can be used by just intuitively understanding them.

In [2]:
:t setField

setField :: forall (w :: WidgetType) (f :: Field). (RElem f (WidgetFields w) (RIndex f (WidgetFields w)), ToPairs (Attr f), IHaskellWidget (IPythonWidget w)) => IPythonWidget w -> SField f -> FieldType f -> IO ()

The `setField` function takes three arguments:

1. A widget
2. A `singleton` for a `Field`
3. A value for the `Field`

In [3]:
:t getField

getField :: forall (w :: WidgetType) (f :: Field). RElem f (WidgetFields w) (RIndex f (WidgetFields w)) => IPythonWidget w -> SField f -> IO (FieldType f)

The `getField` function takes a `Widget`, and a singleton for a `Field` and returns the value of that `Field` for the `Widget`.

## Buttons

Let's play with buttons as a starting example:

In [4]:
button <- mkButton     -- Construct a button

In [5]:
button                 -- Display the button

In [6]:
-- The button widget has many properties.
properties button >>= mapM_ print

ModelModule
ModelName
ViewModule
ViewName
MsgThrottle
Version
OnDisplayed
Visible
CSS
DOMClasses
Width
Height
Padding
Margin
Color
BackgroundColor
BorderColor
BorderWidth
BorderRadius
BorderStyle
FontStyle
FontWeight
FontSize
FontFamily
Description
Tooltip
Disabled
Icon
ButtonStyle
ClickHandler

You can double click to the left of a cell's output to hide it.

The properties shown above are actually types, and we have to use singletons to pass them to `setField`. Let's try making the button widget wider.

In [7]:
-- 250 pixels wide
setField button SWidth 250

There is a lot that can be customized. For example:

In [8]:
setField button SDescription "Click Me (._.\")"
setField button SButtonStyle SuccessButton
setField button SBorderStyle RidgeBorder
setField button SBorderWidth 20
setField button SBorderRadius 30
setField button SPadding 10
setField button SHeight 125
setField button SFontFamily "cursive"
setField button SFontSize 30

The button widget also provides a click handler. We can make it do anything, except console input.

In [9]:
setField button SClickHandler $ putStrLn "fO_o"

## The String Widgets

These widgets are used to represent string-y data. There are four of them. Let's create one of each:

In [10]:
htm <- mkHTMLWidget
latex <- mkLatexWidget
text <- mkTextWidget
area <- mkTextArea

Widgets can be displayed explicitly using `display` from `IHaskell.Display`. Multiple displays of the same widget result in the same view being displayed multiple times. These views always stay synchronized as they are represent the same object.

In [11]:
import IHaskell.Display (display)

In [12]:
display htm
display latex
display text
display area

The `String` widgets have a `StringValue` property which represents their string content. We can modify it to display text using these widgets.

In [13]:
setField htm SStringValue "<b>Bold</b>"

In [14]:
setField latex SStringValue "$x + y$"
-- The default width is somewhat small
setField latex SWidth 400

To add some space around the widgets, we can use the `Padding` field.

In [15]:
setField htm SPadding 10
setField latex SPadding 10
setField text SPadding 10
setField area SPadding 10

The `TextWidget` and `TextArea` also have a `Placeholder` property, which represents the text displayed in empty widgets.

In [16]:
setField text SPlaceholder "Enter your text here..."
setField area SPlaceholder "Parsed output will appear here..."

`TextWidget` and `TextArea` also accept input. The `StringValue` of the widget is automatically updated on every change to the widget. Additionally, the `TextWidget` also has a `SubmitHandler` which is triggered on hitting the return/enter key.

The example below sets up the `TextWidget` and `TextArea` for parsing numbers using parsec. The `TextWidget` is used to recieve input, and the `TextArea` is used to display output.

In [17]:
import Text.Parsec
import Text.Parsec.String
import Data.Text (pack, unpack)
import Control.Applicative ((<$>))

------------------------- THE PARSER ---------------------------

-- Parse a single digit
digit :: Parser Char
digit = oneOf ['0'..'9']

-- Parse a multi-digit number.
number :: Parser Integer
number = do
  digits <- many1 digit -- At least one digit
  return (read digits)  -- Convert [Char] to Integer
  
-- Parse a country code, starting with a +.
countryCode :: Parser Integer
countryCode = do
  char '+'
  number
  
-- Parse an area code, optionally with parentheses.
areaCode :: Parser Integer
areaCode = choice [withParens, withoutParens]
  where
    withParens = between (char '(') (char ')') withoutParens
    withoutParens = number
  
-- Simple data type representing a phone number.
-- Real phone numbers are much more complex!
data PhoneNumber = PhoneNumber {
    phoneCountryCode :: Maybe Integer,
    phoneNumbers :: [Integer]
  } deriving (Eq, Show)
  
phoneNumber :: Parser PhoneNumber
phoneNumber = do
  -- Try to parse a country code. If it doesn't work, it's Nothing.
  c <- optionMaybe countryCode
  optional separator
  a1 <- areaCode
  separator -- Separator required after area code
  a2 <- number
  separator -- Separator required before last group of digits
  a3 <- number
  return (PhoneNumber c [a1, a2, a3])
  
  where
    separator = oneOf " -"

----------------------- WIDGET HANDLING ------------------------

setField text SChangeHandler $ do
  input <- unpack <$> getField text SStringValue
  str <- case parse phoneNumber "<text widget>" input of
             Left error -> return (show error)
             Right x -> return (show x)
  setField area SStringValue (pack str)

## More widgets

There are a lot of widgets. To prevent repeating the same thing again and again, some short examples involving rest of the other widgets are shown below.

### The `Bool` Widgets

#### `CheckBox` and `ToggleButton`

These widgets can be used to represent a Boolean value.

In [18]:
-- Using toggle buttons to represent a 8-bit binary number.
import Control.Monad (replicateM, forM_)
import Data.IORef
import Data.Text (pack, unpack)
import IHaskell.Display (plain)

val <- newIORef 0

sgn <- mkCheckBox -- Create a check box
tgs <- replicateM 8 mkToggleButton -- Create a toggle button
let tbuttons = zip tgs $ iterate (*2) 1

-- Output widget to display output
o <- mkOutputWidget

setField sgn SDescription "Negative"
setField sgn SChangeHandler $ do
  modifyIORef val negate
  readIORef val >>= replaceOutput o . plain . show

setField o SWidth 200

forM_ tbuttons $ \(t, n) -> do
  setField t SDescription "0"
  setField t SChangeHandler $ do
    f <- getField t SBoolValue
    setField t SDescription (if f then "1" else "0")
    modifyIORef val (if f then (+n) else (\x->x-n))
    readIORef val >>= replaceOutput o . plain . show

sgn -- Display the check box
mapM_ display tgs -- Display the toggle buttons
o -- Display the output widget

### The `Output` Widget

The output widget can be used to display rich output. `IHaskell.Display` provides functions to create such rich displays from raw data.

In [19]:
import IHaskell.Display
:t plain
:t html
:t jpg
:t svg
:t latex
:t javascript
:t IHaskell.Display.many

plain :: String -> DisplayData

html :: String -> DisplayData

jpg :: Width -> Height -> Base64 -> DisplayData

svg :: String -> DisplayData

latex :: LatexWidget

javascript :: String -> DisplayData

IHaskell.Display.many :: [Display] -> Display

The `Output` widget is meant to be used through the functions:

+ `appendOutput`: Append more output to the widget.
+ `clearOutput`: Clear the output widget ASAP.
+ `clearOutput_`: Clear the output widget on next use of `appendOutput`.
+ `replaceOutput`: Clear then append.

In [20]:
o <- mkOutputWidget -- Make output widget
setField o SWidth 500
o -- Display output widget
appendOutput o $ IHaskell.Display.html "<font color=\"WHITE\"><marquee direction=\"left\" style=\"background:RED\">The <b>OUTPUT</b> Widget</marquee></font>"
appendOutput o $ IHaskell.Display.html "<font color=\"WHITE\"><marquee direction=\"left\" style=\"background:BLUE\">Is really <b>SIMPLE</b></marquee></font>"
appendOutput o $ IHaskell.Display.html "<font color=\"WHITE\"><marquee direction=\"left\" style=\"background:GREEN\">Use it as an <b>UPDATABLE</b> display</b></marquee></font>"

### The `Image` Widget

This widget can be used to display images given in the form of base64 encoded `Text`. The widget has a `B64Value` field, which can be changed to display images to it. It also has an `ImageFormat` field, which is set to `PNG` by default.

In [21]:
:t IHaskell.Display.base64
:t IHaskell.Display.encode64

IHaskell.Display.base64 :: ByteString -> Base64

IHaskell.Display.encode64 :: String -> Base64

In [22]:
-- Uncomment the line below to install HTTP if you don't have it
-- :!cabal install HTTP
import Network.HTTP

get url = simpleHTTP (getRequest url) >>= getResponseBody
jpg <- get "http://imgs.xkcd.com/comics/functional.png"

img <- mkImageWidget
setField img SB64Value (IHaskell.Display.encode64 jpg)
img

### The `Selection` Widgets

These widgets can be used to select one from many. The `SelectMultiple` widget allows multiple selections, whereas `Dropdown`, `RadioButtons`, `ToggleButtons`, and `Select` only allow one selection.

In [23]:
-- Allows single selection
tgbs <- mkToggleButtons

-- Allows multiple selections
msel <- mkSelectMultiple

In [24]:
setField msel SDescription "Functions to show (One or more)"
setField msel SOptions (OptionLabels ["sin", "cos"])

setField tgbs SDescription "Plot style"
setField tgbs SOptions (OptionLabels ["line", "point"])

In [25]:
import Graphics.Rendering.Chart.Easy hiding (tan)
import Graphics.Rendering.Chart.Backend.Cairo
import qualified Data.ByteString as B

import Control.Monad (when, forM)
import Data.Maybe (fromJust)

dset :: [(String, [(Double, Double)])]
dset = [("sin", zmap sin r), ("cos", zmap cos r)]
  where zmap f xs = zip xs (map f xs)
        r = [0, 0.1 .. 6.3]

i <- mkImageWidget
setField i SWidth 500
setField i SHeight 500

-- Redraw the plot based on values from the widgets
refresh = do
  -- Read values from the widgets
  funs <- map unpack <$> getField msel SSelectedValues
  sty <- unpack <$> getField tgbs SSelectedValue
  
  let pts = zip funs (map (fromJust . flip lookup dset) funs)
      opts = def { _fo_size = (500, 500) }
  toFile opts ".chart" $ do
    layout_title .= "Plotting: " ++ unwords funs
    if sty == "line"
      then mapM_ (\(s, ps) -> plot (line s [ps])) pts
      else mapM_ (\(s, ps) -> plot (points s ps)) pts

  img <- B.readFile ".chart"
  setField i SB64Value (base64 img)
  
-- Add event handlers to make widgets work
setField msel SSelectionHandler refresh
setField tgbs SSelectionHandler refresh

In [26]:
-- Display the widgets
msel
tgbs
i

The `Dropdown`, `RadioButtons` and `Select` widgets behave just like the `ToggleButtons` widget. They have the same properties, and the same functionality.

### The Numeric Widgets

**NOTE**: The following examples use widgets with `Int` in their names. There are also analogous widgets with `Float` in their names.

As the widgets are the same operationally, only the `Int` widgets are shown.

#### `IntText` and `BoundedIntText`

In [27]:
int <- mkIntText
bit <- mkBoundedIntText

In [28]:
int
bit

Both the widgets are similar, but the second one possesses some additional properties.

In [29]:
setField bit SMaxInt 20
setField bit SMinInt 10
setField bit SChangeHandler (getField bit SIntValue >>= print)

Now, the first widget will accept arbitrary input whereas the second one wil accept input the the 10-20 range. For example, try entering large values and hitting return/enter in the second widget.

#### `IntSlider` and `IntRangeSlider`

Both these widgets are sliders (duh!). `IntSlider` represents a single value, whereas `IntRangeSlider` represents a pair (range) of values.

In [30]:
ins <- mkIntSlider
irs <- mkIntRangeSlider

In [31]:
ins
irs

In [32]:
getField irs SIntPairValue

(25,75)

#### `IntProgress`

This widget is meant to be used as a progress bar.

In [33]:
inp <- mkIntProgress
inp

In [34]:
setField inp SIntValue 42